In [ ]:

import heapq
import math
import random

dictionary_set = set()          # Data structure: Set ensures uniqueness
vowels = {'a', 'e', 'i', 'o', 'u'}
failed_paths = set()

# Load dictionary
def load_dictionary(file_name):
    global dictionary_set
    try:
        with open(file_name, "r") as file:
            dictionary_set = set(word.strip().lower() for word in file)
    except FileNotFoundError:
        print("Error: Dictionary file not found.")
        dictionary_set = set()

# Check if a word is in the dictionary
def is_word_in_dictionary(word):
    word = word.lower()
    return word in dictionary_set

# Find path between 2 words in the dictionary
def find_word_path(starting_word, goal_word, search_method, detail_output):
    starting_word = starting_word.lower()
    goal_word = goal_word.lower()
    load_dictionary("dictionary.txt")
    if not is_word_in_dictionary(starting_word) or not is_word_in_dictionary(goal_word):
        print("Error: One or both words not in the dictionary.")
        return

    if search_method == 1:
        path = a_star_search(starting_word, goal_word)
        if path:
            for i, word in enumerate(path):
                print(word)
                if detail_output and i == 1:   # Print heuristic if detail_output=True
                    heuristic_value = heuristic(word, goal_word)
                    print(f"Heuristic: {heuristic_value}")
        else:
            print("No path found")

    if search_method == 2:
        path = hill_climbing_search(starting_word, goal_word)
        if path:
            for i, word in enumerate(path):
                print(word)
        else:
            print("No path found")


    if search_method == 3:
        path = simulated_annealing_search(starting_word, goal_word, detail_output)
        if path:
            for i, word in enumerate(path):
                print(word)
        else:
            print("No path found")


    if search_method == 4:
        path = local_beam_search(starting_word, goal_word, detail_output)
        if path:
            for i, word in enumerate(path):
                print(word)
        else:
            print("No path found")

    if search_method == 5:
        path = genetic_algorithm_search(starting_word, goal_word, detail_output)
        if path:
            for i, word in enumerate(path):
                print(word)
        else:
            print("No path found")

def a_star_search(start, goal):
    global failed_paths
    frontier = []
    heapq.heappush(frontier, (0, (start, 0)))  # Start with a tuple containing word and cost
    explored = set()
    came_from = {}
    g_cost = {start: 0}
    iterations = 0

    while frontier:

        if iterations > 100000000:   # Limit iterations to ensure termination
            print("Exceeded maximum iterations.")
            return None
        iterations += 1

        _, current = heapq.heappop(frontier)
        if current[0] == goal:
            return reconstruct_path(came_from, start, current[0])

        explored.add(current[0])

        if current[0] in failed_paths:
            continue

        for neighbor, cost in get_neighbors(current, explored):
            if neighbor in failed_paths:
                continue

            tentative_g_cost = g_cost[current[0]] + cost

            if neighbor not in g_cost or tentative_g_cost < g_cost[neighbor]:
                came_from[neighbor] = current[0]
                g_cost[neighbor] = tentative_g_cost
                f_cost = tentative_g_cost + heuristic(neighbor, goal)
                heapq.heappush(frontier, (f_cost, (neighbor, cost)))

    return None

def hill_climbing_search(start, goal, max_iteration=5):

    current_word = (start, 0)  # Start as a tuple with cost 0
    counter = 0
    explored = set()
    path = [start]

    while counter < max_iteration:
        while current_word[0] != goal:

            explored.add(current_word[0])

            # Get neighbors and their costs
            neighbors = get_neighbors(current_word, explored)

            if not neighbors:
                break


            next_word = choose_best_neighbor(neighbors, goal)

            # Stop if no improvement
            if heuristic(next_word[0], goal) >= heuristic(current_word[0], goal):
                break

            current_word = next_word
            path.append(current_word[0])

        if current_word[0] == goal:
            return path

        counter += 1

        restart_neighbors = get_neighbors(start, explored)

        if restart_neighbors:

            current_word = choose_best_neighbor(restart_neighbors, goal)
            path = [start]
        else:

            break

    return None

def get_neighbors(word, explored):
    if isinstance(word, tuple):
        word = word[0]  # Extract the actual word if a tuple is passed

    neighbors = []

    # Replace one letter
    for i in range(len(word)):
        for char in "abcdefghijklmnopqrstuvwxyz":
            if char != word[i]:
                new_word = word[:i] + char + word[i+1:]
                if new_word not in explored and new_word in dictionary_set:
                    cost = 0.25 if char in vowels else 1  # Pricing based on the cost of change
                    neighbors.append((new_word, cost))

    # Add one letter
    for i in range(len(word) + 1):
        for char in "abcdefghijklmnopqrstuvwxyz":
            new_word = word[:i] + char + word[i:]
            if new_word not in explored and new_word in dictionary_set:
                cost = 0.25 if char in vowels else 1
                neighbors.append((new_word, cost))

    # Remove one letter
    for i in range(len(word)):
        new_word = word[:i] + word[i+1:]
        if new_word not in explored and new_word in dictionary_set:
            cost = 0.25 if word[i] in vowels else 1
            neighbors.append((new_word, cost))

    return neighbors

def choose_best_neighbor(neighbors, goal):
    heap = []
    for neighbor, cost in neighbors:
        heuristic_value = heuristic(neighbor, goal)
        total_cost = cost + heuristic_value  # Combine cost and heuristic
        heapq.heappush(heap, (total_cost, (neighbor, cost)))

    _, best_neighbor = heapq.heappop(heap)
    return best_neighbor

def heuristic(current_word, goal_word):
    if isinstance(current_word, tuple):  # Extract word if tuple is passed
        current_word = current_word[0]

    len_current = len(current_word)
    len_goal = len(goal_word)
    i, j = 0, 0
    cost = 0

    while i < len_current and j < len_goal:
        if current_word[i] == goal_word[j]:   # Comparison of characters by position.
            i += 1
            j += 1
        else:
            cost += 0.25 if goal_word[j] in vowels else 1
            i += 1
            j += 1

    while i < len_current:
        cost += 0.25 if current_word[i] in vowels else 1
        i += 1

    while j < len_goal:
        cost += 0.25 if goal_word[j] in vowels else 1
        j += 1

    return cost

def reconstruct_path(came_from, start, current):
    path = []
    while current in came_from:
        path.append(current)
        current = came_from[current]
    path.append(start)
    return path[::-1]

def simulated_annealing_search(start, goal, detail_output, max_iterations=10000):
    explored = set()
    path = [start]
    current = start
    transition_chance = 0

    for t in range(0,max_iterations):
        T = schedule(t)

        if T <= 0.001:
            break

        next_word = pick_neighbor(current,goal, explored)
        if not next_word:
            break

        delta_E = heuristic(current, goal) - heuristic(next_word, goal)

        if delta_E > 0:
            current = next_word
            path.append(current)
            explored.add(current)
            if (t==0 and detail_output) :
                print (f"next word: {next_word}, probability: 1")

        else:
            transition_chance = math.exp(delta_E / T) if T > 0 else 0
            if (t==0 and detail_output) :
                print (f"next word: {next_word}, probability: {transition_chance}")

            if random.random() < transition_chance:
                current = next_word
                path.append(current)
                explored.add(current)

        if current == goal:
            return path

    return None

def schedule(t, max_temp=100, cooling_rate=0.01):
    return max_temp / (1 + cooling_rate * t)

def pick_neighbor(word, goal, explored):
    neighbors = get_neighbors(word, explored)
    if not neighbors:
        return None


    neighbors_with_heuristics = [(n[0], heuristic(n[0], goal)) for n in neighbors]


    min_heuristic = min(h for _, h in neighbors_with_heuristics)
    best_neighbors = [(n, h) for n, h in neighbors_with_heuristics if h <= min_heuristic + 1]


    if best_neighbors:
        weights = [1 / (h + 1e-6) for _, h in best_neighbors]
        total_weight = sum(weights)
        probabilities = [w / total_weight for w in weights]
        chosen_neighbor = random.choices([n for n, _ in best_neighbors], weights=probabilities, k=1)[0]
        return chosen_neighbor


    random_neighbor = random.choice(neighbors)
    return random_neighbor[0]

def local_beam_search(start, goal, detail_output, k=3, max_iterations=10000):

    current_beams = [{"heuristic": heuristic(start, goal), "word": start}]
    paths = {start: [start]}
    iterations = 0

    while iterations < max_iterations:
        iterations += 1

        for beam in current_beams:
            if beam["word"] == goal:
                return paths[beam["word"]]

        all_neighbors = []
        neighbor_words = []  # Helper list for words only
        for beam in current_beams:
            neighbors = get_neighbors(beam["word"], set())
            neighbor_words.extend([neighbor[0] for neighbor in neighbors])

            if detail_output and iterations == 1:
                 print(f"Neighbors: {neighbor_words}")
            for neighbor in neighbors:
                heuristic_value = heuristic(neighbor, goal)
                if neighbor not in paths:
                    all_neighbors.append({"heuristic": heuristic_value, "word": neighbor, "parent": beam["word"]})
                    paths[neighbor] = paths[beam["word"]] + [neighbor]

        if not all_neighbors:
            break


        # Select the top k neighbors
        all_neighbors.sort(key=lambda x: x["heuristic"])
        current_beams = all_neighbors[:k]

        if detail_output and iterations == 1:
            chosen_actions = [{"word": beam["word"]} for beam in current_beams]
            print(f"Chosen actions: {chosen_actions}")

    return None

def genetic_algorithm_search(start, goal, detail_output, population_size=10, max_generations=10000, mutation_probability=0.2):
    population = [{"current_word": start, "path": [start]} for _ in range(population_size)]

    first_change = True
    for _ in range(max_generations):
        fitness_scores = [fitness(path, goal) for path in population]

        if any(path["current_word"] == goal for path in population):
            return next(path["path"] for path in population if path["current_word"] == goal)

        total_fitness = sum(fitness_scores)
        probabilities = [score / total_fitness for score in fitness_scores]
        selected_parents = random.choices(population, probabilities, k=population_size)

        new_population = []
        for i in range(0, len(selected_parents), 2):
            parent1 = selected_parents[i]
            parent2 = selected_parents[(i + 1) % len(selected_parents)]
            child = crossover(parent1, parent2)
            if random.random() < mutation_probability:
                child = mutate(child)
            new_population.append(child)

        population = new_population

        if detail_output and first_change:
            print("Population after first change:")
            for individual in population:
                print(individual)
            first_change = False


    return None
def fitness(individual, goal):
    return 1 / (heuristic(individual["current_word"], goal) + len(individual["path"]))

def mutate(individual):
    current_word = individual["current_word"]
    neighbors = get_neighbors(current_word, set())
    if neighbors:
        new_word = random.choice(neighbors)[0]
        return {"current_word": new_word, "path": individual["path"] + [new_word]}
    return individual

def crossover(parent1, parent2):
    common_path = set(parent1["path"]).intersection(parent2["path"])
    if common_path:
        crossover_point = random.choice(list(common_path))
        index1 = parent1["path"].index(crossover_point)
        index2 = parent2["path"].index(crossover_point)
        new_path = parent1["path"][:index1+1] + parent2["path"][index2+1:]
        return {"current_word": new_path[-1], "path": new_path}
    return parent1


if __name__ == "__main__":

    # [EN Note 1: Setup] Load the dictionary once at startup to prepare for all searches.
    load_dictionary("dictionary.txt")

    print("--- Word Ladder Solver (Advanced Search Algorithms) ---")

    # 1. Get user input
    try:
        start_word = input("Enter starting word: ").lower()
        goal_word = input("Enter goal word: ").lower()

        print("\nSelect Search Algorithm:")
        print("1: A* Search")
        print("2: Hill Climbing Search")
        print("3: Simulated Annealing Search")
        print("4: Local Beam Search")
        print("5: Genetic Algorithm Search")

        search_method = int(input("Enter choice (1-5): "))
        detail_output_choice = input("Show detailed output (y/n)? ").lower()
        detail_output = detail_output_choice == 'y'

    except ValueError:
        print("Invalid input. Please enter numbers for the search method.")
        exit()

    # 2. Validate inputs
    if search_method not in range(1, 6):
        print("Invalid search method selected.")
        exit()

    # 3. Run the search
    print(f"\n--- Running Search Method {search_method} ---")


Error: Dictionary file not found.
--- Word Ladder Solver (Advanced Search Algorithms) ---
Enter starting word: banana
Enter goal word: ban

Select Search Algorithm:
1: A* Search
2: Hill Climbing Search
3: Simulated Annealing Search
4: Local Beam Search
5: Genetic Algorithm Search
Enter choice (1-5): 2
Show detailed output (y/n)? y

--- Running Search Method 2 ---
